In [1]:
from keras.applications import VGG16
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model
from sklearn.metrics.pairwise import euclidean_distances

%matplotlib inline

Using TensorFlow backend.


Get the list of images

In [2]:
shoe_list = !ls ZShoes/Images/

In [3]:
len(shoe_list)

25257

In [4]:
shoe_number = [int(x.split('.')[0]) for x in shoe_list]
shoe_number.sort()

In [5]:
shoe_number[:5]

[1, 2, 3, 4, 5]

Setup the VGG16 Model

In [6]:
# load VGG16
#Get back the convolutional part of a VGG network trained on ImageNet
model = VGG16(weights='imagenet', include_top=True)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
def image_pre_process(img_file):
    img = imread(img_file)
    img = resize(img, (224, 224), preserve_range=True).astype(np.float32)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [9]:
model2 = Model(input=model.input, output=model.get_layer('fc2').output) # gets the output in fc2

/Users/robertomac/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  if __name__ == '__main__':


Function that can get the data in chunks

In [10]:
def process_data(start, end):
    out1 = []
    for i in range(start,end):
        print(i)
        name = 'ZShoes/Images/' + str(i) + '.jpg'
        im = image_pre_process(name)
        out = model2.predict(im)
        out1.append(out[0])
    return out1

In [11]:
# if file has already started it starts on next shoe, otherwise, at the beginning
list_names = !ls
if 'Features.csv' in list_names:
    df = pd.read_csv('Features.csv')
    shoe_num_start = df.shape[0]
else:
    shoe_num_start = 1
shoe_num_start

25257

In [15]:
val = 10

while shoe_num_start<=len(shoe_number):
# for i in range(shoe_num_start,shoe_num_start+val):#range(1,int(np.ceil(len(shoe_number)/val))):
    start = shoe_num_start
    end = min(start+val,len(shoe_number))
    print(start,end)
    out1 = process_data(start, end)
    res = pd.DataFrame(out1)
    res['shoe_number'] = pd.DataFrame(list(range(start,end)))
    res.set_index('shoe_number', inplace=True)

    if shoe_num_start==1:
        res.to_csv('Features.csv')
    else:
        with open('Features.csv', 'a') as f:
            res.to_csv(f, header = False)
    shoe_num_start+=val
    shoe_num_start = min(shoe_num_start,len(shoe_number))

25257 25258
25257


/Users/robertomac/anaconda/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


25257 25258
25257
25257 25258
25257
25257 25258
25257
25257 25258
25257
25257 25258
25257
25257 25258
25257
25257 25258
25257
25257 25258
25257


KeyboardInterrupt: 